In [4]:
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

variant_table = pd.read_csv('input.txt', sep='\t')
variant_table
included = variant_table[(variant_table['Exclude?']=='No') | (variant_table['Exclude?']=='NO_NEW')][['ID', 'Exclude?', 'Exclude_Reason', 'narrative']].drop_duplicates()
print(len(included), 'included')
success_table = variant_table[(variant_table['Exclude?']=='No') | (variant_table['Exclude?']=='NO_NEW')][['ID', 'Exclude?', 'Exclude_Reason', 'issues/notes','narrative', 'unfiltered_result', 'filtered_result', 'genomic_coordinates_hg38', 'effect', 'MOI', 'zygosity', 'inheritance', 'Family_Type', 'variant_type']]
print(len(set(success_table['ID'])),'genes')
print(len(success_table), 'variants')

239 included
239 genes
296 variants


In [5]:
success_table = pd.read_csv('fig2_input.tsv', sep='\t')

,ID,Exclude?,Exclude_Reason,issues/notes,narrative,unfiltered_result,filtered_result,genomic_coordinates_hg38,effect,MOI,...,Variant_Level_noMOI_maxPathSource_noN_filtered_15_85_human_ppi_revel_mvp_noWL,Variant_Level_noMOI_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_rank_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_p_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_variant_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_pheno_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_combined_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_maxPath_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Type_noN_filtered_15_85_all_models_revel_mvp_noWL,Variant_Level_noMOI_maxPathSource_noN_filtered_15_85_all_models_revel_mvp_noWL
0,UDN001459_SLC20A2,No,.,NaN,_single het; SNV,Exomiser_Success,Exomiser_Success,chr8:42437579 T>C,splice_acceptor,AD,...,NONE,Variant_Present_noMOI,1.0,0.0001,1.0000,0.6977,0.9588,NaN,splice_acceptor_variant,NONE
1,UDN004009_CTSA,No,.,NaN,_compound het; SNV; SNV,Exomiser_Success,Exomiser_Success,chr20:45892776 GA>G,frameshift,AR,...,NONE,Variant_Present_noMOI,2.0,0.0006,0.9987,0.6140,0.9043,NaN,frameshift_variant,NONE
2,UDN004009_CTSA,No,.,NaN,_compound het; SNV; SNV,Exomiser_Success,Exomiser_Success,chr20:45898442 C>G,missense,AR,...,MVP,Variant_Present_noMOI,2.0,0.0006,0.9945,0.6140,0.9043,0.994943,missense_variant,MVP
3,UDN008634_GDF11,No,.,NaN,_single het; SNV,Exomiser_Success,Exomiser_Success,chr12:55749666 C>G,stop_gained,AD,...,NONE,Variant_Present_noMOI,1.0,0.0007,1.0000,0.6713,0.9465,NaN,stop_gained,NONE
4,UDN011317_STT3A,No,.,NaN,_single het; SNV,Exomiser_Success,Exomiser_Success,chr11:125614163 A>G,missense,AD,...,MVP,Variant_Present_noMOI,2.0,0.0147,0.9451,0.5401,0.7322,0.945090,missense_variant,MVP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,UDN994504_H4C5,No,.,NaN,_single het; SNV,Exomiser_Success,Exomiser_Success,chr6:26204939 T>C,missense,AD,...,MVP,Variant_Present_noMOI,19.0,0.4145,0.1126,0.8369,0.0278,0.112649,missense_variant,MVP
292,UDN995589_BAP1,No,.,NaN,_single het; SNV,Exomiser_Success,Exomiser_Success,chr3:52408062 A>G,missense,AD,...,REVEL,Variant_Present_noMOI,1.0,0.0001,0.9840,0.6406,0.9174,0.984000,missense_variant,REVEL
293,UDN061684_MRTFB,NO_NEW,.,FOUND IN RESEARCH REQUEST; gene previously MLK...,_single het; SNV,Exomiser_Success,Exomiser_Success,chr16:14213578 C>G,missense,AD,...,MVP,Variant_Present_noMOI,7.0,0.0199,0.9889,0.5001,0.7293,0.988852,missense_variant,MVP
294,UDN348954_H4C5,NO_NEW,.,gene alias HIST1H4E,_single het; SNV,Exomiser_Success,Exomiser_Success,chr6:26204780 C>T,NaN,AD,...,MVP,Variant_Present_noMOI,26.0,0.4821,0.2924,0.5935,0.0117,0.292424,missense_variant,MVP


In [6]:
def create_df(highest_rank, compare, successTable):
    x = range(1,highest_rank)
    denom = len(successTable)
    data=[]
    for runtype in compare:
        success = successTable.loc[(successTable['Variant_Level_noMOI_'+str(runtype)]=='Variant_Present_noMOI') | (successTable['Variant_Level_noMOI_'+str(runtype)]=='Variant_Present_noMOI')]
        for i in x:
            num = len(success.loc[success['Variant_Level_noMOI_rank_'+str(runtype)] <= i])
            data.append([i,num, (num/denom)*100, runtype])
        print('highest rank:', max(success['Variant_Level_noMOI_rank_'+str(runtype)]), 'max%:', (num/denom)*100, num)

    df = pd.DataFrame(data, columns=['Rank', 'NumPatients', 'Percent_Variants', 'Run_Type'])
    return df, denom

In [7]:
def create_plot(denom, source, compare, color_scheme, domain):
    title=str(denom) +'Diagnostic Variants (Variant Level - no MOI requirement)'
    bigChart = alt.Chart(source, title=title).mark_line().encode(
        x=alt.X('Rank', title='Exomiser Rank of Causal Variant'),
        y=alt.Y('Percent_Variants', title='Percent of Causal Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
        color=alt.Color('Run_Type:N',legend=None,sort=compare,scale=alt.Scale(domain=domain,scheme=color_scheme)),
        tooltip=['Rank', 'Percent_Variants', 'Run_Type']
    ).properties(
        width=600,
        height=500)
    
    zoom_source = source.loc[source['Rank'] <=30]

    zoomChart = alt.Chart(zoom_source, title=title).mark_line(point=alt.OverlayMarkDef(size=50)).encode(
        x=alt.X('Rank', title='Exomiser Rank of Causal Variant'),
        y=alt.Y('Percent_Variants', title='Percent of Causal Variants within Exomiser Rank', scale=alt.Scale(domain=[0,100])),
        color=alt.Color('Run_Type:N', sort=domain, scale=alt.Scale(domain=domain,scheme=color_scheme)),
        tooltip=['Rank', 'Percent_Variants', 'Run_Type']
    ).properties(
        width=500,
        height=400)
    plot=alt.vconcat(bigChart, zoomChart)#.configure_legend(labelLimit=0).configure_axis(
        # labelFontSize=15,
        # titleFontSize=15).configure_legend(labelLimit=0,labelFontSize=15, titleFontSize=15)#.configure_axis(grid=False)
    plot=alt.vconcat(zoomChart)
    return plot


## A. Algorithms for phenotypic similarity

In [37]:
domain= ['noN_filtered_15_85_all_models_revel_mvp_noWL','noN_filtered_15_85_mouse_revel_mvp_alphaM_noWL','noN_filtered_15_85_phenixPrioritiser_revel_mvp_noWL','noN_filtered_15_85_noPhenoPrior_revel_mvp_noWL','', 'noN_filtered_15_85_human_ppi_revel_mvp_noWL','b','c', 'noN_filtered_15_85_human_revel_mvp_noWL']#'noN_filtered_15_85_human_REMM_revel_mvp_PSF0.501_noWL_genomiser','noN_filtered_15_85_all_models_REMM_revel_mvp_PSF0.501_noWL_genomiser']
run_types = ['noN_filtered_15_85_mouse_revel_mvp_alphaM_noWL','noN_filtered_15_85_all_models_revel_mvp_noWL', 'noN_filtered_15_85_phenixPrioritiser_revel_mvp_noWL','noN_filtered_15_85_noPhenoPrior_revel_mvp_noWL' ]
##PHIVE, hiPHIVE, PhenIX, no prioritization
run_types = ['noN_filtered_15_85_mouse_revel_mvp_alphaM_noWL','noN_filtered_15_85_all_models_revel_mvp_noWL', 'noN_filtered_15_85_phenixPrioritiser_revel_mvp_noWL','noN_filtered_15_85_noPhenoPrior_revel_mvp_noWL','noN_filtered_15_85_human_revel_mvp_noWL','noN_filtered_15_85_human_ppi_revel_mvp_noWL','noN_filtered_15_85_human_ppi_mouse_revel_mvp_noWL']

a_df,denom = create_df(300, run_types, success_table)
plot_1a = create_plot(denom, a_df, run_types, 'category20', domain)

# plot_1a.configure_legend(labelLimit=0).configure_axis(
#         labelFontSize=15,
#         titleFontSize=15).configure_legend(labelLimit=0,labelFontSize=15, titleFontSize=15)

highest rank: 431.0 max%: 96.62162162162163 286
highest rank: 218.0 max%: 96.95945945945947 287
highest rank: 256.0 max%: 96.95945945945947 287
highest rank: 346.0 max%: 96.28378378378379 285
highest rank: 277.0 max%: 96.95945945945947 287
highest rank: 454.0 max%: 96.62162162162163 286
highest rank: 218.0 max%: 96.95945945945947 287


In [38]:
plot_1a.configure_axis(grid=True,
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=15, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')


alt.VConcatChart(...)